In [5]:
import os

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,validation_curve,cross_val_score
from sklearn.datasets.samples_gen erator import make_blobs
from sklearn.metrics import r2_score, accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

# Regularization on a Linear model

Same case study as the Linear model. We're dealing with the Life Expectancy dataset. 

In [3]:
#finding path, ugh don't ask!
path, file = os.path.split(os.path.abspath("Life_Expectancy_Data.csv"))
path = path+'\data\\'

#loading data
data = pd.read_csv(path+file)

target = 'Life expectancy '
cols = ['Life expectancy ',
      'Alcohol', 'percentage expenditure', 'Hepatitis B',
       ' BMI ',  'Polio', 'Total expenditure',
       'Diphtheria ',  'GDP', 
       'Income composition of resources', 'Schooling']
data.dropna(inplace=True)

#splitting data
X_train, X_test, y_train, y_test = train_test_split(data[cols[1:]], data[target], random_state = 2, test_size =0.2)

#create model
reg = LinearRegression().fit(X_train, y_train)

#making predictions
y_pred = reg.predict(X_test)

## without regularization

In [4]:
#validate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error: {}'.format(mse))
print('R-squared Error: {}'.format(r2))
print('Cross validation Score: ',cross_val_score(reg, X_train, y_train, cv=5))

Mean Squared Error: 28.989551051546545
R-squared Error: 0.6542775615619447
Cross validation Score:  [0.64104672 0.63257221 0.5829629  0.6613591  0.56388255]


## with Regularization

# Regularization on Polynomial Regression

We'll use the same case study in Polynomial Regression with the COVID-19 dataset

In [6]:
#finding path, ugh don't ask!
path, file = os.path.split(os.path.abspath("covid_19_data.csv"))
path = path+'\data\\'

#loading data
data = pd.read_csv(path+file)

#rename pesky column names
data.rename(columns = {'Province/State': 'State','Country/Region':'Country'}, inplace=True)

#dropping unneccessary columns
data.drop(labels=['SNo','Last Update','State'], axis=1, inplace=True)

#changing dtype of date column
data['ObservationDate']=pd.to_datetime(data['ObservationDate'])

#grouping stats for each day
grouped = data.groupby(['Country','ObservationDate'])

#creating new df
df = pd.DataFrame(columns =data.columns)
country =[]
date=[]
conf =[]
rec=[]
dead =[]
for groupinfo, stats in grouped:
    country.append(groupinfo[0])
    date.append(groupinfo[1])
    rec.append(np.sum(stats['Recovered']))
    dead.append(np.sum(stats['Deaths']))
    conf.append(np.sum(stats['Confirmed']))

df['ObservationDate'] =date 
df['Country'] = country
df['Confirmed'] = conf
df['Deaths'] = dead
df['Recovered']= rec

#create label encoder instance
le = LabelEncoder()

#fit into labelencoder and get labels
labels = le.fit_transform(df['Country'])

#set labels to country
df['Country'] = labels

#splitting data
features = ['Country','Confirmed','Deaths']
target = 'Recovered'
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size =0.2, random_state=0)

#create polynomial features
poly = PolynomialFeatures(degree =2 )

#fit X_train
X_poly = poly.fit_transform(X_train)

#create and fit into linear model
poly_reg = LinearRegression().fit(X_poly, y_train)

#make predictions
y_pred = poly_reg.predict(poly.transform(X_test))

## Without regularization

In [7]:
#validate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error: {}'.format(mse))
print('R-squared Error: {}'.format(r2))

Mean Squared Error: 190869184.0756124
R-squared Error: 0.8984295871261034


## With regularization

# Regularization on Logistic Model